## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Lincoln,AU,-34.7333,135.8667,75.92,45,4,16.60,clear sky
1,1,Cidreira,BR,-30.1811,-50.2056,70.90,95,11,3.85,few clouds
2,2,Cape Town,ZA,-33.9258,18.4232,59.92,87,0,11.01,clear sky
3,3,Provideniya,RU,64.3833,-173.3000,32.36,80,20,2.24,few clouds
4,4,Urumqi,CN,43.8010,87.6005,52.75,58,20,4.47,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Cidreira,BR,-30.1811,-50.2056,70.90,95,11,3.85,few clouds
7,7,Lafayette,US,30.2335,-92.0668,67.95,62,0,0.00,clear sky
16,16,Sakakah,SA,29.9697,40.2064,71.76,36,0,4.54,clear sky
17,17,Ouadda,CF,8.0777,22.4007,70.90,91,100,2.59,overcast clouds
30,30,Puerto Ayora,EC,-0.7393,-90.3518,71.73,92,11,5.41,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Cidreira,BR,-30.1811,-50.2056,70.90,95,11,3.85,few clouds
7,7,Lafayette,US,30.2335,-92.0668,67.95,62,0,0.00,clear sky
16,16,Sakakah,SA,29.9697,40.2064,71.76,36,0,4.54,clear sky
17,17,Ouadda,CF,8.0777,22.4007,70.90,91,100,2.59,overcast clouds
30,30,Puerto Ayora,EC,-0.7393,-90.3518,71.73,92,11,5.41,few clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Cidreira,BR,70.90,few clouds,-30.1811,-50.2056,
7,Lafayette,US,67.95,clear sky,30.2335,-92.0668,
16,Sakakah,SA,71.76,clear sky,29.9697,40.2064,
17,Ouadda,CF,70.90,overcast clouds,8.0777,22.4007,
30,Puerto Ayora,EC,71.73,few clouds,-0.7393,-90.3518,
31,Bambous Virieux,MU,73.22,few clouds,-20.3428,57.7575,
34,New Norfolk,AU,70.92,broken clouds,-42.7826,147.0587,
35,Yambio,SS,67.15,overcast clouds,4.5721,28.3955,
37,Hobart,AU,71.26,few clouds,-42.8794,147.3294,
39,Busselton,AU,61.52,broken clouds,-33.6500,115.3333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except:
        print ("Cannot find hotel... skipping.")

Cannot find hotel... skipping.
Cannot find hotel... skipping.
Cannot find hotel... skipping.
Cannot find hotel... skipping.
Cannot find hotel... skipping.
Cannot find hotel... skipping.
Cannot find hotel... skipping.
Cannot find hotel... skipping.
Cannot find hotel... skipping.
Cannot find hotel... skipping.
Cannot find hotel... skipping.
Cannot find hotel... skipping.
Cannot find hotel... skipping.
Cannot find hotel... skipping.
Cannot find hotel... skipping.
Cannot find hotel... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.replace(r'^\s*$', np.nan, regex=True).dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Cidreira,BR,70.90,few clouds,-30.1811,-50.2056,Hotel Castelo
7,Lafayette,US,67.95,clear sky,30.2335,-92.0668,"Drury Inn & Suites Lafayette, LA"
16,Sakakah,SA,71.76,clear sky,29.9697,40.2064,Fakhamat Al Orjoana Apartment
30,Puerto Ayora,EC,71.73,few clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
31,Bambous Virieux,MU,73.22,few clouds,-20.3428,57.7575,Casa Tia Villa
...,...,...,...,...,...,...,...
678,Coahuayana,MX,74.12,clear sky,18.7333,-103.6833,El rranchito michuacan de ocampos
683,Kundiawa,PG,74.08,overcast clouds,-6.0167,144.9667,Pari
686,Armeria,MX,74.32,clear sky,18.9333,-103.9667,Hotel la Herradura
688,Guigue,VE,73.24,overcast clouds,10.0853,-67.7792,La vasquereña


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))